In [ ]:
import pandas as pd

data_jan18 = pd.read_csv("yellow_tripdata_2018-01.csv")

#CHECK HOW TO LOAD JUST SPECIFIC COLLUMS
#THINK HOW TO LOAD FILES TO BROWSE BETWEEN MONTHS
#CHECK HOW MANY ODD VALUES THAT DON'T MAKE SENSE IN THE DATA SET


In [6]:
data_jan18.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,4.5,0.5,0.5,0.00,0.0,0.3,5.80
1,1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,1,N,239,140,2,14.0,0.5,0.5,0.00,0.0,0.3,15.30
2,1,2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,1,N,262,141,1,6.0,0.5,0.5,1.00,0.0,0.3,8.30
3,1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,1,N,140,257,2,33.5,0.5,0.5,0.00,0.0,0.3,34.80
4,1,2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,1,N,246,239,1,12.5,0.5,0.5,2.75,0.0,0.3,16.55


In [ ]:
datajan18.describe()

In [ ]:
datajan18.dtypes

In [3]:
data_zone = pd.read_csv("taxi _zone_lookup.csv")

In [4]:
data_zone.head()

,LocationID,Borough,Zone,service_zone
0,1,EWR,Newark Airport,EWR
1,2,Queens,Jamaica Bay,Boro Zone
2,3,Bronx,Allerton/Pelham Gardens,Boro Zone
3,4,Manhattan,Alphabet City,Yellow Zone
4,5,Staten Island,Arden Heights,Boro Zone


In [24]:
#df.join(df2.set_index('id1'), on='id')
data_merge = pd.merge(data_jan18, data_zone,how = "left", left_on='PULocationID', right_on = "LocationID")

In [25]:
data_merge.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,LocationID,Borough,Zone,service_zone
0,1,2018-01-01 00:21:05,2018-01-01 00:24:23,1,0.5,1,N,41,24,2,...,0.5,0.5,0.00,0.0,0.3,5.80,41,Manhattan,Central Harlem,Boro Zone
1,1,2018-01-01 00:44:55,2018-01-01 01:03:05,1,2.7,1,N,239,140,2,...,0.5,0.5,0.00,0.0,0.3,15.30,239,Manhattan,Upper West Side South,Yellow Zone
2,1,2018-01-01 00:08:26,2018-01-01 00:14:21,2,0.8,1,N,262,141,1,...,0.5,0.5,1.00,0.0,0.3,8.30,262,Manhattan,Yorkville East,Yellow Zone
3,1,2018-01-01 00:20:22,2018-01-01 00:52:51,1,10.2,1,N,140,257,2,...,0.5,0.5,0.00,0.0,0.3,34.80,140,Manhattan,Lenox Hill East,Yellow Zone
4,1,2018-01-01 00:09:18,2018-01-01 00:27:06,2,2.5,1,N,246,239,1,...,0.5,0.5,2.75,0.0,0.3,16.55,246,Manhattan,West Chelsea/Hudson Yards,Yellow Zone


[RQ1] In what period of the year Taxis are used more? tpep_pickup_datetime or tpep_dropoff_datetime
[RQ2] What are the time slots with more passengers? tpep_pickup_datetime and tpep_dropoff_datetime
[RQ3] Do the all trips last the same? tpep_pickup_datetime and tpep_dropoff_datetime
[RQ4] What is the most common way of payments? payment_type
[RQ5] Does a long distance correlate with the duration of the trip on average? trip_distance

Used: tpep_pickup_datetime,	tpep_dropoff_datetime, trip_distance, PULocationID, payment_type